# Deployment playground

## 1. Loading the trained model for inference

In this step, we set up the path to the `src/` directory so we can import custom modules like `predictor.py`.  
This allows us to use the `predict_fraud()` function defined in our pipeline, which loads the trained model and applies preprocessing.

In [ ]:
# Setup src path and import predict_fraud
# ─── Core ─────────────────────────────────────────────
import os
import sys
import pandas as pd

# ─── Add project src directory to sys.path ─────────────
# Navigate one level up from the notebook (notebooks/ → root/)
notebook_dir = os.path.abspath("")  # current working dir where notebook runs
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)


# ─── Custom imports ─────────────────────────────────
from predictor import predict_fraud

## 2. Creating a sample input for the API

We take a single row from the processed dataset `train_final_ready.csv` and remove the `isFraud` target column.  
Then we convert it to a dictionary compatible with our API input format, and optionally save it as a JSON file.

In [2]:
sample = pd.read_csv("../data/processed/train_final_ready.csv").sample(1, random_state=42)

**example input for the API**

In [ ]:
import json

# Load a row from the training set
df = pd.read_csv("../data/processed/train_final_ready.csv").drop(columns=["isFraud"])
sample = df.sample(1, random_state=42)

# Convert sample to JSON-compatible dictionary and optionally save to file
sample_json = sample.to_dict(orient="records")[0]

# Save the sample JSON to a file (optional)
with open("request_example.json", "w") as f:
    json.dump(sample_json, f, indent=2)

# Show the JSON object
import pprint
pprint.pprint(sample_json)

{'C1': 3.0,
 'C10': 0.0,
 'C11': 1.0,
 'C12': 0.0,
 'C13': 2.0,
 'C14': 2.0,
 'C2': 1.0,
 'C3': 0.0,
 'C4': 0.0,
 'C5': 0.0,
 'C6': 1.0,
 'C7': 0.0,
 'C8': 0.0,
 'C9': 2.0,
 'D1': 0.0,
 'D10': 145.0,
 'D11': 0.0,
 'D12': 0.0,
 'D13': 0.0,
 'D14': 0.0,
 'D15': 145.0,
 'D2': 97.0,
 'D3': 8.0,
 'D4': 145.0,
 'D5': 0.0,
 'D6': 0.0,
 'D7': 0.0,
 'D8': 37.875,
 'D9': 0.6666659712791443,
 'DeviceInfo': 1598,
 'DeviceType': 0,
 'M1': 1,
 'M2': 1,
 'M3': 0,
 'M4': 0,
 'M5': 0,
 'M6': 1,
 'M7': 0,
 'M8': 0,
 'M9': 1,
 'P_emaildomain': 2,
 'ProductCD': 4,
 'R_emaildomain': 16,
 'TransactionAmt': 724.0,
 'TransactionDT': 12153579.0,
 'TransactionID': 3457624.0,
 'V1': 1.0,
 'V10': 0.0,
 'V100': 0.0,
 'V101': 0.0,
 'V102': 0.0,
 'V103': 0.0,
 'V104': 1.0,
 'V105': 1.0,
 'V106': 1.0,
 'V107': 1.0,
 'V108': 1.0,
 'V109': 1.0,
 'V11': 0.0,
 'V110': 1.0,
 'V111': 1.0,
 'V112': 1.0,
 'V113': 1.0,
 'V114': 1.0,
 'V115': 1.0,
 'V116': 1.0,
 'V117': 1.0,
 'V118': 1.0,
 'V119': 1.0,
 'V12': 1.0,
 'V120': 1.

## 3. Getting a prediction from the model

We pass the sample row to the `predict_fraud()` function, which:
- Applies the same preprocessing steps as during training
- Uses the trained model to predict whether the transaction is fraudulent
- Returns a prediction (`0` or `1`) and the associated probability

In [3]:
result = predict_fraud(sample)
print(result)

{'prediction': 0, 'fraud_probability': np.float64(0.012)}


## 4. Send request to API 

If the FastAPI server is running locally (via `uvicorn src.main:app --reload`),  
you can send a POST request to test the model deployed as an API.

In [10]:
import requests

response = requests.post(
    "http://127.0.0.1:8000/predict",
    json=sample_json
)
print(response.json())

{'prediction': 0, 'fraud_probability': 0.012}


## 5. Conclusion and Next Steps

With this notebook, we've successfully demonstrated how to:

- Load a trained machine learning model for fraud detection.
- Prepare a valid input transaction in JSON format.
- Predict fraud using both a Python function and a deployed FastAPI endpoint.

### ✅ Next Steps

- Include this notebook in your documentation to show how the model can be consumed by other systems.
- Add unit tests for the `predict_fraud` function using sample inputs.
- (Optional) Automate this example as part of an integration test pipeline or DAG.
- Consider adding logging and versioning for API predictions.

👉 This notebook serves as a bridge between offline modeling and real-world deployment.